In [1]:
import numpy as np
import h5py   
import math
import matplotlib.pyplot as plt  
from numba import jit
from numba import f4, f8, u8


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def onemove_in_cube_true(p0,v):
    htime=np.abs((np.floor(p0)-p0+(v>0))/v)
    minLoc=htime.argmin()
    dist=htime[minLoc]
    htime=p0+dist*v
    htime[minLoc]=np.round(htime[minLoc])+np.spacing(np.abs(np.float64(htime[minLoc])))*np.sign(v[minLoc])
    return htime, dist

In [3]:
def onemove_in_cube_true_ut(p0,v):
    htime=np.abs((np.floor(p0)-p0+(v>0))/v)
    minLoc=htime.argmin()
    dist=htime[minLoc]
    htime=p0+dist*v
    htime[minLoc]=np.round(htime[minLoc])+np.spacing(np.abs(np.float64(htime[minLoc])))*np.sign(v[minLoc])
    return htime, dist

In [4]:
Nx = 208
Ny = 256
Nz = 225
Mx = 20
My = 20
D = 2
h = 50
H = h + Nz + 200
orginOffset = np.array(
        [(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 + (Ny / 2), 0], dtype='float64')
ep = np.array([Nx / 2, Ny / 2, H], dtype = 'float64')
muBone = 0.573
muFat = 0.193
orginOffset = np.array([(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 + (Ny / 2), 0]) 
ep = np.array([Nx / 2, Ny / 2, H])                                              
f = h5py.File('headct.h5', 'r')
headct=np.array(f.get('ct'))
mu=np.zeros((Nx,Ny,Nz))
mu[np.nonzero(mu>0)]=((headct[np.nonzero(mu>0)]-0.3)/(0.7))*(muBone-muFat)+muFat
del headct
detector=np.zeros((Mx,My))
pos=np.zeros(3)
dir=np.zeros(3)
#%timeit run_loop(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset)

In [5]:
def run_loop(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset):
    for z in prange(Mx*My): 
        j=z%Mx
        i=int(z/Mx) 
        #print(str(i)+' '+str(j))
        pos=np.array([orginOffset[0]*i+D,orginOffset[1]+D*j, 0])
        dir=np.array((ep-pos)/np.linalg.norm(ep-pos))
        L=1
        #print('Dir'+str(dir))
        while pos[2]< h+Nz:
            pos,dist=onemove_in_cube_true(pos,dir)
            if 0 <= pos[0] < Nx and 0<=pos[1]<Ny  and h<=pos[2] < h+Nz:
                L=L*np.exp(-1*mu[math.floor(pos[0]),math.floor(pos[1]),math.floor(pos[2]-h)]*dist)
        detector[i][j] = L;

In [8]:
def run_loop_p(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset):
    for z in range(0,Mx*My): 
        j=z%Mx
        i=int(z/Mx) 
        #print(str(i)+' '+str(j))
        pos=np.array([orginOffset[0]*i+D,orginOffset[1]+D*j, 0])
        dir=np.array((ep-pos)/np.linalg.norm(ep-pos))
        L=1
        #print('Dir'+str(dir))
        while pos[2]< h+Nz:
            htime=np.abs((np.floor(pos)-pos+(dir>0))/dir)
            minLoc=htime.argmin()
            dist=htime[minLoc]
            htime=pos+dist*dir
            htime[minLoc]=np.round(htime[minLoc])+np.spacing(np.abs(np.float64(htime[minLoc])))*np.sign(dir[minLoc])
            if 0 <= pos[0] < Nx and 0<=pos[1]<Ny  and h<=pos[2] < h+Nz:
                L=L*np.exp(-1*mu[math.floor(pos[0]),math.floor(pos[1]),math.floor(pos[2]-h)]*dist)
        detector[i][j] = L;

In [11]:
run_loop_p.inspect_types()

AttributeError: 'function' object has no attribute 'inspect_types'

In [12]:
jit('void(f8[:,:],f8[:],f8[:],u8,u8,u8,u8,u8,f8[:,:,:],f8[:],f8[:])',nopython=True)(run_loop_p).inspect_types()

TypingError: Failed at nopython (nopython frontend)
array(float64, 1d, C) not allowed in a homogenous sequence
[1] During: resolving callee type: Function(<built-in function array>)
[2] During: typing of call at <ipython-input-8-1fd9168db799> (7)

In [ ]:
%timeit run_loop_p(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset)

In [ ]:
%timeit run_loop(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset)

In [15]:
def testA(detector,pos,dir,Mx,My,h,Nz,D,mu,ep,orginOffset):
    for z in range(0,Mx*My): 
        j=z%Mx
        i=int(z/Mx) 
        #print(str(i)+' '+str(j))
        pos=np.array([orginOffset[0]*i+D,orginOffset[1]+D*j, 0])
        dir=np.array((ep-pos)/np.linalg.norm(ep-pos))
        L=1

In [17]:
jit('void(f8[:,:],f8[:],f8[:],u8,u8,u8,u8,u8,f8[:,:,:],f8[:],f8[:])',nopython=True)(testA).inspect_types()

TypingError: Failed at nopython (nopython frontend)
array(float64, 1d, C) not allowed in a homogenous sequence
[1] During: resolving callee type: Function(<built-in function array>)
[2] During: typing of call at <ipython-input-15-c8fdb351be0f> (7)